In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={
}

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, NotebookSubstepRunResult, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
   
    inputs=[
        {STEP_NAME: 'model_eval', ENTITY_NAME: 'reports_model_eval_ipynb'
        #ENTITY_PATH: "/data/home/jovyan/example_median_house_price/test/model_eval/*/reports_model_eval_ipynb"
        }
        
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
import re
from os import path as osp

In [ ]:
# run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
# read inputs 

inputs = substep.inputs(step_name="model_eval")

#inputs.reports_model_eval_ipynb metrics.json

all_metrics_path = re.sub(r'run-[-0-9]*', '*', inputs.reports_model_eval_ipynb)
all_metrics_path = osp.join(all_metrics_path, 'metrics.json')
print(all_metrics_path)

metrics = spark.read.json(all_metrics_path)

In [ ]:
metrics.printSchema()

In [ ]:
df = metrics.select('experiment_name', 'eval_result.r2', 'eval_result.rmse', 'run_id', 'version.bento_service_version')

In [ ]:
df.show()

In [ ]:
# stop spark
SinaraSpark.stop_session()